In [ ]:
import cv2
import numpy as np
from sklearn.decomposition import PCA
import umap
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
import os
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA



In [ ]:
 مسیر تصاویر
input_folder = r"D:\HACKATOON\preimage"  # مسیر پوشه تصاویر

# تعریف متدهای استخراج ویژگی (به همان صورت قبلی)
def method_1(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist_h = cv2.calcHist([hsv_image], [0], None, [32], [0, 256]).flatten()
    hist_s = cv2.calcHist([hsv_image], [1], None, [32], [0, 256]).flatten()
    hist_v = cv2.calcHist([hsv_image], [2], None, [32], [0, 256]).flatten()
    return np.hstack([hist_h, hist_s, hist_v])

# تابع استخراج ویژگی‌های هندسی
def extract_geometric_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresholded = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    features = []
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        width = np.linalg.norm(box[0] - box[1])
        height = np.linalg.norm(box[1] - box[2])
        aspect_ratio = width / height if height != 0 else 0  # جلوگیری از تقسیم بر صفر

        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
        else:
            cx, cy = 0, 0

        angle = rect[2]
        features.append([area, perimeter, aspect_ratio, cx, cy, angle])

    # اطمینان از یکسان بودن تعداد ویژگی‌ها برای تمام تصاویر
    max_features = 10  # مثلا فرض کنیم حداکثر 10 ویژگی هندسی برای هر تصویر داریم
    features = features[:max_features]  # اگر تعداد بیشتر از 10 است، فقط 10 ویژگی اول را نگه داریم
    while len(features) < max_features:
        features.append([0] * 6)  # اگر تعداد کمتر از 10 است، ویژگی‌های صفر به آن اضافه می‌کنیم

    return features

# ترکیب ویژگی‌های هندسی با سایر ویژگی‌ها
def extract_all_features(image):
    color_features = method_1(image)
    geometric_features = extract_geometric_features(image)
    # ترکیب ویژگی‌های رنگ با هندسی
    combined_features = np.hstack([color_features, np.array(geometric_features).flatten()])
    return combined_features

# استخراج ویژگی‌ها از تصاویر
extracted_features = []
image_paths = []  # ذخیره مسیر تصاویر برای نمایش
for filename in os.listdir(input_folder):
    if filename.endswith((".png", ".jpg", ".jpeg")):
        file_path = os.path.join(input_folder, filename)
        image = cv2.imread(file_path)

        if image is not None:  # فقط در صورتی که تصویر به درستی بارگذاری شده باشد
            image = cv2.resize(image, (128, 128))  # تغییر اندازه تصویر
            # استخراج ویژگی‌ها
            image_features = extract_all_features(image)
            extracted_features.append(image_features)
            image_paths.append(file_path)  # اضافه کردن مسیر تصویر به لیست
        else:
            print(f"Warning: Image {file_path} not loaded correctly!")

features = np.array(extracted_features)

# کاهش ابعاد با UMAP
umap_reducer = umap.UMAP(n_components=23, random_state=42, n_jobs=-1)  # از تمام رشته‌های CPU استفاده می‌کند
umap_reduced = umap_reducer.fit_transform(features)

# خوشه‌بندی با KMeans
clustering_model = KMeans(n_clusters=5, random_state=42)  # تعداد خوشه‌ها
labels = clustering_model.fit_predict(umap_reduced)

# نمایش تصاویری از خوشه‌ها
def show_images_with_labels(image_paths, labels, num_samples=5):
    unique_labels = np.unique(labels)

    for label in unique_labels:
        # انتخاب نمونه‌هایی از هر خوشه
        cluster_indices = np.where(labels == label)[0]

        # اصلاح نام متغیر figure
        plt.figure(figsize=(15, 15))  # نمایش به صورت یک figure جدید
        plt.title(f"Cluster {label}")

        for i, idx in enumerate(cluster_indices[:num_samples]):
            image_path = image_paths[idx]
            image = cv2.imread(image_path)

            if image is not None:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # تبدیل رنگ به RGB برای matplotlib
                plt.subplot(1, num_samples, i + 1)
                plt.imshow(image)
                plt.axis('off')
                plt.title(f"Cluster {label}")
            else:
                print(f"Error: Unable to load image at {image_path}")

        plt.show()

# نمایش تصاویری از خوشه‌ها با تعداد محدود
show_images_with_labels(image_paths, labels, num_samples=5)  # 5

In [ ]:


# مسیر تصاویر
input_folder = r"D:\HACKATOON\preimage"  # مسیر پوشه تصاویر

# متدهای استخراج ویژگی
feature_methods = [
    "Method 1: Color Histogram Only",
    "Method 2: Color, Texture, Edge",
    "Method 3: Color, Texture, Advanced Edge"
]

# متدهای کاهش ابعاد
reduction_methods = {'PCA': PCA, 'UMAP': umap.UMAP}

# تعداد مؤلفه‌ها برای کاهش ابعاد
n_components_list_method_1_2 = [2, 5, 10, 50]  # برای متد ۱ و ۲
n_components_list_method_3 = [2, 10, 100, 500, 600, 700, 800]  # برای متد ۳

# متدهای خوشه‌بندی
clustering_methods = {
    "K-Means": KMeans(n_clusters=5, random_state=42),
    "Spectral Clustering": SpectralClustering(n_clusters=5, affinity='nearest_neighbors', random_state=42),
    "Gaussian Mixture": GaussianMixture(n_components=5, random_state=42)
}

# بهترین ترکیب و مقدار سیلوئت
best_config = None
best_silhouette = -1

# تعریف متدهای استخراج ویژگی (به همان صورت قبلی)
def method_1(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist_h = cv2.calcHist([hsv_image], [0], None, [32], [0, 256]).flatten()
    hist_s = cv2.calcHist([hsv_image], [1], None, [32], [0, 256]).flatten()
    hist_v = cv2.calcHist([hsv_image], [2], None, [32], [0, 256]).flatten()
    return np.hstack([hist_h, hist_s, hist_v])

def method_2(image):
    color_features = extract_color_features(image)
    texture_features = extract_texture_features(image)
    edge_features = extract_edge_features(image)
    combined_features = np.hstack([color_features, texture_features, edge_features])
    return combined_features

def method_3(image):
    color_features = extract_color_features(image)
    texture_features = extract_texture_features(image)
    edge_and_intensity_features = extract_edge_and_intensity_features(image)
    combined_features = np.hstack([color_features, texture_features, edge_and_intensity_features])
    return combined_features

# تابع استخراج ویژگی‌های هندسی
def extract_geometric_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresholded = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    features = []
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        width = np.linalg.norm(box[0] - box[1])
        height = np.linalg.norm(box[1] - box[2])
        aspect_ratio = width / height if height != 0 else 0  # جلوگیری از تقسیم بر صفر

        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
        else:
            cx, cy = 0, 0

        angle = rect[2]
        features.append([area, perimeter, aspect_ratio, cx, cy, angle])

    # اطمینان از یکسان بودن تعداد ویژگی‌ها برای تمام تصاویر
    max_features = 10  # مثلا فرض کنیم حداکثر 10 ویژگی هندسی برای هر تصویر داریم
    features = features[:max_features]  # اگر تعداد بیشتر از 10 است، فقط 10 ویژگی اول را نگه داریم
    while len(features) < max_features:
        features.append([0] * 6)  # اگر تعداد کمتر از 10 است، ویژگی‌های صفر به آن اضافه می‌کنیم

    return features

# ترکیب ویژگی‌های هندسی با سایر ویژگی‌ها
def extract_all_features(image, method):
    if method == "Method 1: Color Histogram Only":
        color_features = method_1(image)
        geometric_features = extract_geometric_features(image)
        # ترکیب ویژگی‌های رنگ با هندسی
        combined_features = np.hstack([color_features, np.array(geometric_features).flatten()])
        return combined_features
    elif method == "Method 2: Color, Texture, Edge":
        combined_features = method_2(image)
        geometric_features = extract_geometric_features(image)
        # ترکیب ویژگی‌ها با ویژگی‌های هندسی
        combined_features = np.hstack([combined_features, np.array(geometric_features).flatten()])
        return combined_features
    elif method == "Method 3: Color, Texture, Advanced Edge":
        combined_features = method_3(image)
        geometric_features = extract_geometric_features(image)
        # ترکیب ویژگی‌ها با ویژگی‌های هندسی
        combined_features = np.hstack([combined_features, np.array(geometric_features).flatten()])
        return combined_features
    else:
        return []


# شروع Grid Search
for method_name in feature_methods:
    print(f"\nEvaluating feature extraction method: {method_name}")

    # استخراج ویژگی‌ها
    extracted_features = []
    for filename in os.listdir(input_folder):
        if filename.endswith((".png", ".jpg", ".jpeg")):
            file_path = os.path.join(input_folder, filename)
            image = cv2.imread(file_path)
            if image is not None:
                image = cv2.resize(image, (128, 128))
                # استخراج ویژگی‌ها با توجه به متد انتخابی
                image_features = extract_all_features(image, method_name)
                geometric_features = extract_geometric_features(image)
                # ترکیب ویژگی‌های هندسی با سایر ویژگی‌ها
                combined_features = np.hstack([image_features, np.array(geometric_features).flatten()])
                extracted_features.append(combined_features)
    features = np.array(extracted_features)

    # انتخاب تعداد مؤلفه‌ها بر اساس متد
    if method_name == "Method 3: Color, Texture, Advanced Edge":
        n_components_list = n_components_list_method_3
    else:
        n_components_list = n_components_list_method_1_2

    for reduction_name, reduction_class in reduction_methods.items():
        for n_components in n_components_list:
            print(f"  Reduction: {reduction_name}, Components: {n_components}")

            if reduction_name == "PCA":
                reducer = reduction_class(n_components=n_components)
            else:
                reducer = reduction_class(n_components=n_components, random_state=42, n_jobs=-1)

            reduced_features = reducer.fit_transform(features)

            for clustering_name, clustering_model in clustering_methods.items():
                print(f"    Clustering: {clustering_name}")

                # خوشه‌بندی
                labels = clustering_model.fit_predict(reduced_features)

                # محاسبه Silhouette Coefficient
                silhouette_avg = silhouette_score(reduced_features, labels)
                print(f"      Silhouette Coefficient: {silhouette_avg:.4f}")

                # به‌روزرسانی بهترین ترکیب
                if silhouette_avg > best_silhouette:
                    best_silhouette = silhouette_avg
                    best_config = {
                        'Feature Method': method_name,
                        'Reduction Method': reduction_name,
                        'n_components': n_components,
                        'Clustering Method': clustering_name,
                        'Silhouette Coefficient': silhouette_avg
                    }

# نمایش بهترین ترکیب
print("\nBest Configuration:")
for key, value in best_config.items():
    print(f"{key}: {value}")